In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Data

In [1]:
!pip install gdown

In [2]:
!gdown --id 1CnQ_-vlJ6ITPTnHksTdSWQgHaApNRLwR

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1CnQ_-vlJ6ITPTnHksTdSWQgHaApNRLwR
From (redirected): https://drive.google.com/uc?id=1CnQ_-vlJ6ITPTnHksTdSWQgHaApNRLwR&confirm=t&uuid=23a8170a-3807-4e66-b02e-f2fa845421e8
To: /kaggle/working/norm.csv
100%|███████████████████████████████████████| 2.50G/2.50G [00:19<00:00, 127MB/s]


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('./norm.csv')

In [5]:

import csv
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional,GlobalAveragePooling1D
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
tf.config.experimental.set_visible_devices(tf.config.experimental.list_physical_devices('GPU'), 'GPU')

# Splitting the Datset

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
df.columns

Index(['Unnamed: 0', 'review_text', 'rating', 'lower', 'stop_words',
       'stemming', 'lemma'],
      dtype='object')

In [12]:
X=df['lemma'].astype(str)
Y=df['rating']
X_train, X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=0)
X_train, X_valid,y_train,y_valid = train_test_split(X_train,y_train,test_size=0.2,random_state=0)

In [13]:
vocab_size = 5000 
embedding_dim = 60
max_length = 250
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<oov>' # OOV = Out of Vocabulary

> num_words=This will be the maximum number of words from our resulting tokenized data vocabulary which are to be used

> oov_token=This is the token which will be used for out of vocabulary tokens encountered during the tokenizing 

> max_length = maximum length after padding

![Token Image](https://www.kdnuggets.com/wp-content/uploads/text-tokens-tokenization-manning.jpg)

> word_index = which maps words in our vocabulary to their numeric representation, a mapping which will be essential for encoding our sequences. 

In [15]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

validation_sequences = tokenizer.texts_to_sequences(X_valid)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

> The first thing we have to convert text data to numbers therefore we use different approaches like
* one hot encoding
* word embedding
> Word embeddings give us a way to use an efficient, dense representation in which similar words have a similar encoding.An embedding is a dense vector of floating point values (the length of the vector is a parameter you specify). Instead of specifying the values for the embedding manually, they are trainable parameters (weights learned by the model during training, in the same way a model learns weights for a dense layer).
1.  input_length = This parameter specifies the length of the input sequences. It's the number of words in each input sequence
2.  output_dim = This parameter defines the dimensionality of the dense embedding.

**GlobalAveragePooling1D**
> The GlobalAveragePooling1D layer returns a fixed-length output vector for each example by averaging over the sequence dimension. This allows the model to handle input of variable length, in the simplest way possible.

**Bidirectional LSTM**
> This will create two copies of the hidden layer, one fit in the input sequences as-is and one on a reversed copy of the input sequence. By default, the output values from these LSTMs will be concatenated.
> That means that instead of the layer receiving 250 input and  giving 60 output, it will now receive 250 inputs  of 120 (60 units + 60 units) outputs.

> 

In [16]:
model = Sequential()

model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(embedding_dim,return_sequences=True)))
model.add(GlobalAveragePooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax')) #output layer

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 60)           300000    
                                                                 
 dropout (Dropout)           (None, 250, 60)           0         
                                                                 
 bidirectional (Bidirection  (None, 250, 120)          58080     
 al)                                                             
                                                                 
 global_average_pooling1d (  (None, 120)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 64)                7744      
                                                                 
 dense_1 (Dense)             (None, 6)                 3

> **Adam optimization** is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
this method is "computationally efficient, has little memory requirement, invariant to diagonal rescaling of gradients, and is well suited for problems that are large in terms of data/parameters".

> loss='**sparse_categorical_crossentropy**': This sets the loss function for the training. sparse_categorical_crossentropy is commonly used for classification tasks when your labels are integers (e.g., 0, 1, 2) and not one-hot encoded.

> metrics='**accuracy**': During training, the model will calculate and display the accuracy as one of the metrics. The accuracy metric provides a measure of how well the model is performing on the training data.

In [17]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [20]:
num_epochs = 10
history = model.fit(train_padded, y_train, batch_size=512,epochs=num_epochs, validation_data=(validation_padded, y_valid), verbose=1)

Epoch 1/10
788/788 [==============================] - 45s 57ms/step - loss: 0.9791 - accuracy: 0.5817 - val_loss: 1.0101 - val_accuracy: 0.5671
Epoch 2/10
788/788 [==============================] - 44s 56ms/step - loss: 0.9745 - accuracy: 0.5843 - val_loss: 0.9979 - val_accuracy: 0.5726
Epoch 3/10
788/788 [==============================] - 44s 56ms/step - loss: 0.9675 - accuracy: 0.5874 - val_loss: 1.0133 - val_accuracy: 0.5670
Epoch 4/10
788/788 [==============================] - 44s 55ms/step - loss: 0.9637 - accuracy: 0.5886 - val_loss: 1.0005 - val_accuracy: 0.5697
Epoch 5/10
788/788 [==============================] - 44s 56ms/step - loss: 0.9597 - accuracy: 0.5903 - val_loss: 0.9945 - val_accuracy: 0.5736
Epoch 6/10
788/788 [==============================] - 44s 56ms/step - loss: 0.9547 - accuracy: 0.5929 - val_loss: 0.9964 - val_accuracy: 0.5730
Epoch 7/10
788/788 [==============================] - 44s 56ms/step - loss: 0.9517 - accuracy: 0.5931 - val_loss: 0.9946 - val_accuracy:

In [21]:
results = model.evaluate(test_padded, y_test)

3938/3938 [==============================] - 45s 11ms/step - loss: 0.9917 - accuracy: 0.5753


In [22]:
!gdown --id 1a0Wr8NPC5ysRuMZ9nDJVciV9QCFC_5Qw

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1a0Wr8NPC5ysRuMZ9nDJVciV9QCFC_5Qw
From (redirected): https://drive.google.com/uc?id=1a0Wr8NPC5ysRuMZ9nDJVciV9QCFC_5Qw&confirm=t&uuid=9aa3d8c8-e6bf-40f1-b212-d04ba2bada47
To: /kaggle/working/updatednormtest.csv
100%|█████████████████████████████████████████| 755M/755M [00:03<00:00, 208MB/s]


# **Test Dataset**

In [23]:
test = pd.read_csv("updatednormtest.csv")

In [24]:
test.head()

,Unnamed: 0,review_text,review_id,lower,stop_words
0,0,"I'm going to keep this review short, because I...",d76ce5becf493e5c653610edb806ffc4,i am going to keep this review short because i...,going keep review short feel like dangerous to...
1,1,"You know, I was really stoked to see this come...",9c5c9aed79255a1a610dfc153ee90ad6,you know i was really stoked to see this come ...,know really stoked see come library website re...
2,2,This is one of those books where you know you ...,ad26332459cda8f40581fef7a29b800c,this is one of those books where you know you ...,one books know want reread reading years ago d...
3,3,The perks of being a wallflower \n What does a...,5bb75768be7f2ddcd632336778b67f5a,the perks of being a wallflower what does a wa...,perks wallflower wallflower mean question edgi...
4,4,"So, I wrote a review for this when I read it b...",3d7f76ea566b9ce0700772236094d936,so i wrote a review for this when i read it bu...,wrote review read accidentally deleted whole b...


In [26]:
test_seq = tokenizer.texts_to_sequences(test['stop_words'].astype(str).values)
test_pad = pad_sequences(test_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [27]:
pred = model.predict(test_pad)

8438/8438 [==============================] - 85s 10ms/step


In [32]:
pred

array([[9.8006837e-03, 1.9357001e-04, 8.8076666e-03, 3.4527677e-01,
        5.8718508e-01, 4.8736203e-02],
       [7.6988852e-03, 1.5003842e-01, 5.1094067e-01, 2.5445667e-01,
        6.3920327e-02, 1.2945026e-02],
       [1.3970661e-02, 1.0353695e-04, 5.1208388e-04, 1.2640743e-02,
        2.6515567e-01, 7.0761728e-01],
       ...,
       [2.5294566e-01, 3.5307705e-02, 7.2164997e-02, 1.4172779e-01,
        2.4880546e-01, 2.4904838e-01],
       [1.0822408e-02, 6.4033838e-03, 1.2542748e-01, 5.9451425e-01,
        2.3824731e-01, 2.4585241e-02],
       [8.5124727e-03, 4.9527863e-04, 1.9847502e-03, 1.7274991e-02,
        1.8550859e-01, 7.8622389e-01]], dtype=float32)

In [28]:
predicted_classes = np.argmax(pred, axis=1)

In [30]:
submission = pd.DataFrame ({'review_id':test['review_id'],'rating':predicted_classes})

In [31]:
submission.to_csv('/kaggle/working/updatedsub1.csv',index=False)